# Control code for the SHNO measurement setup

In [ ]:
#import livbraries and necesary fundtions for the script to run
import sys; sys.path.append("C:/Users/SpinAge_SHNO/Documents/Setup_control")
from shno_lib import *
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
import h5py

# Initilize the intruments
magnet = rs_hmp_psu()             # Default COM port
lna_psu = keysight36300()         # Default IP
shno_bias = keithley2450(addr=2)  # IP Connection
vxm = Velmex_VXM()                # Default COM port
fsv = rs_fsv()                    # Default IP  

In [ ]:
# Turn off all power
lna_psu.off()
shno_bias.off()
magnet.off()


In [ ]:
# Control angle. Move to 118 for microscpe clearance.
# note that the angle is move din relative terms
vxm.move_degree(0, speed=2000)

In [ ]:
# set the parameters for manually checking the probe landing
shno_bias.set_output(I=50e-6, V=0.1, turn_on=False)
shno_bias.on()

## Magnetic field sweep

In [ ]:
# initialize the parameteters

# Metadata
sample_name = "mm_L3R4"
date = time.localtime()[1:3]
date = f"{date[0]}-{date[1]}"

# Angle: 82 degrees
bias_current = 8e-3 # SHNO bias: 180nm: 2.2mA, 150nm: 2mA
shno_bias.set_output(I=bias_current, V=2, turn_on=False)

# Set magnet range in output current
start_field = 2     #[A]
stop_field = 4.5      #[A]
step_size = 0.1    #[A]

magnet_range = np.linspace(start_field, stop_field, int(abs((stop_field-start_field)/step_size)+1))
applied_field = np.zeros_like(magnet_range)

# Turn on FSV

start_f = 3E9; stop_f = 9E9
bandwidth = 500E3
capture_points = int(2*(stop_f-start_f)/bandwidth/2)

waveforms = np.zeros((len(magnet_range), capture_points))
waveforms_bg = np.zeros_like(waveforms)
fsv.show_screen(start_f = start_f, stop_f = stop_f, bandwidth=bandwidth, points = capture_points)
frequency = np.linspace(start_f, stop_f, capture_points)



In [ ]:
# Start measurements
lna_psu.lna(state=True)
shno_bias.on()
time.sleep(1)

progress_bar = tqdm(enumerate(magnet_range), total=len(magnet_range))

mag_R = magnet.set_calibrated_current(start_field)

for i, field in progress_bar:
    mag_R = magnet.set_calibrated_current(field, mag_R)
    
    time.sleep(1)
    applied_field[i] = magnet.field_strength()
    progress_bar.set_postfix({'Applied Field': applied_field[i]})
    waveforms[i] = fsv.get_trace(avg_count=50)

# Turn off all power
lna_psu.lna(state=False)
shno_bias.off()
magnet.off()
shno_bias.chirp(0)


In [ ]:
# Save the data
name = f'../Memristor_SHNO_matrix/field_sweep/{date}_{sample_name}_{str(bias_current*1000).replace(".", "_")}mA_83deg.h5'
ans = input(f"Confirm saving? [y/n]:{name}")
if ans == 'y':
    with h5py.File(name, 'w') as f:
        # Store the waveforms, frequency, and applied field
        f.create_dataset('frequency', data=frequency)
        f.create_dataset('waveforms', data=waveforms)
        f.create_dataset('applied_field', data=applied_field)
    print(f"File saved as: '{name}'")
    print("Time:", time.strftime("%H:%M:%S", time.localtime()))
else: 
    print("Save cancelled!")

## Angle Sweep

In [ ]:
# Reset position
vxm.move_degree(0.2)

In [ ]:
# initialize the parameteters

# Metadata
sample_name = "mm_L4R3"
date = time.localtime()[1:3]
date = f"{date[0]}-{date[1]}"

# Angle: 82 degrees
bias_current = 8e-3 # SHNO bias: 180nm: 2.2mA, 150nm: 2mA
shno_bias.set_output(I=bias_current, V=2, turn_on=False)

# Set magnet current 
magnet_current = 3.3 #[A] #3.3A = 0.5 T


# Angle
total_angle = 2
step_size = 0.5
angle_steps = int(total_angle/step_size)
angle_range = np.linspace(0, total_angle, angle_steps)

applied_field = np.zeros(angle_steps)

# Turn on FSV
start_f = 3E9; stop_f = 10E9
bandwidth = 500E3           # noise floor approx -80dBm @ 500e3
capture_points = int(2*(stop_f-start_f)/bandwidth)

waveforms = np.zeros((angle_steps, capture_points))
fsv.show_screen(start_f = start_f, stop_f = stop_f, bandwidth=bandwidth, points = capture_points)
frequency = np.linspace(start_f, stop_f, capture_points)



In [ ]:
# Start measurements
lna_psu.lna(state=True)
shno_bias.on()
time.sleep(1)

mag_R = magnet.set_calibrated_current(magnet_current)

for i in tqdm(range(angle_steps)):
    
    mag_R = magnet.set_calibrated_current(magnet_current, mag_R)

    applied_field[i] = magnet.field_strength()
    waveforms[i] = fsv.get_trace(avg_count=50)
    
    vxm.move_degree(step_size)
    shno_bias.on()
    time.sleep(0.5)

# Fix the too many steps bug
vxm.move_degree(-step_size)

# Turn off all power
lna_psu.lna(state=False)
shno_bias.off()
magnet.off()
shno_bias.chirp(0)

In [ ]:
# Save the data
name = f'../Memristor_SHNO_matrix/angle_sweep/{date}_{sample_name}_{str(bias_current*1000).replace(".", "_")}_startangle_72deg_mA_3_3A.h5'
ans = input(f"Confirm saving? [y/n]:{name}")
if ans == 'y':
    with h5py.File(name, 'w') as f:
        # Store the waveforms, frequency, and applied field
        f.create_dataset('frequency', data=frequency)
        f.create_dataset('waveforms', data=waveforms)
        f.create_dataset('applied_field', data=applied_field)
        f.create_dataset('angle_range', data=angle_range)
    print(f"File saved as: '{name}'")
    print("Time:", time.strftime("%H:%M:%S", time.localtime()))
else: 
    print("Save cancelled!")

## Bias sweep

In [ ]:
# initialize the parameteters

# Metadata
sample_name = "mm_L4R3"
date = time.localtime()[1:3]
date = f"{date[0]}-{date[1]}"

# Angle: 82 degrees

# Set magnet range in output voltage
magnet_field = 3.0#3.3A = 0.5 mT

# Set bias current range
start_bias = 5e-3
stop_bias = 6e-3
bias_step = 0.2e-3

total_steps = int((stop_bias-start_bias)/bias_step)+1
bias_range = np.linspace(start_bias, stop_bias, total_steps)
applied_field = np.zeros_like(bias_range)

# Turn on FSV
start_f = 3E9; stop_f = 9E9
bandwidth = 500E3
capture_points = int(2*(stop_f-start_f)/bandwidth)

waveforms = np.zeros((total_steps, capture_points))
fsv.show_screen(start_f = start_f, stop_f = stop_f, bandwidth=bandwidth, points = capture_points)
frequency = np.linspace(start_f, stop_f, capture_points)


In [ ]:
# Start measurements
lna_psu.lna(state=True)
mag_R = 2 # Initial resistance

for i in tqdm(range(total_steps)):
    mag_R = magnet.set_calibrated_current(magnet_field, mag_R)
    shno_bias.set_output(2, bias_range[i], turn_on=True)
    shno_bias.on()
    time.sleep(1)
    applied_field[i] = magnet.field_strength()
    waveforms[i] = fsv.get_trace(avg_count=50)

# Turn off all power
lna_psu.lna(state=False)
shno_bias.off()
magnet.off()
shno_bias.chirp(0)

In [ ]:
# Save the data
name = f'../Memristor_SHNO_matrix/bias_sweep/{date}_{sample_name}_82deg_field_3_0A.h5'
ans = input(f"Confirm saving? [y/n]:{name}")
if ans == 'y':
    with h5py.File(name, 'w') as f:
        # Store the waveforms, frequency, and applied field
        f.create_dataset('frequency', data=frequency)
        f.create_dataset('waveforms', data=waveforms)
        f.create_dataset('applied_field', data=applied_field)
        f.create_dataset('bias_range', data=bias_range)
    print(f"File saved as: '{name}'")
    print("Time:", time.strftime("%H:%M:%S", time.localtime()))
else: 
    print("Save cancelled!")

## Angle/Field sweep

In [ ]:
vxm.move_degree(2,speed=1600)

In [ ]:
# initialize the parameteters
# Remember to create a folder with the name of the sample!

# Metadata
sample_name = "mm_L4R3"
date = time.localtime()[1:3]
date = f"{date[0]}-{date[1]}"


bias_current = 8.0e-3 # SHNO bias: 180nm: 2.2mA, 150nm: 2mA
shno_bias.set_output(I=bias_current, V=2, turn_on=False)

# Set magnet range in current
start_field = 3.0
stop_field = 4.9
step_size = 0.5

magnet_range = np.linspace(start_field, stop_field, int(abs((stop_field-start_field)/step_size)+1))
applied_field = np.zeros_like(magnet_range)


# Angle
start_angle = 76 # set mnually 

total_angle = 2
step_size = 0.25
angle_steps = int(total_angle/step_size)
angle_range = np.linspace(0, total_angle, angle_steps+1)

# Turn on FSV
start_f = 3E9; stop_f = 9E9
bandwidth =1000E3
capture_points = int(2*(stop_f-start_f)/bandwidth)

waveforms = np.zeros((len(magnet_range), capture_points))
waveforms_bg = np.zeros_like(waveforms)
fsv.show_screen(start_f = start_f, stop_f = stop_f, bandwidth=bandwidth, points = capture_points)
frequency = np.linspace(start_f, stop_f, capture_points)


In [ ]:
# Start measurements
lna_psu.lna(state=True)
shno_bias.on()
time.sleep(1)

for angle in tqdm(angle_range):

    progress_bar = tqdm(enumerate(magnet_range), total=len(magnet_range), leave=False)
    mag_R = magnet.set_calibrated_current(start_field)
    for i, field in progress_bar:
        mag_R = magnet.set_calibrated_current(field, mag_R)
        applied_field[i] = magnet.field_strength()
        progress_bar.set_postfix({'Applied Field': applied_field[i]})
        waveforms[i] = fsv.get_trace(avg_count=50)
        
    # Save the data
    name = f'../Memristor_SHNO_matrix/angle_sweep/field_sweep/{sample_name}/{date}_{str(bias_current*1000).replace(".", "_")}mA_{angle + start_angle}deg.h5'
    with h5py.File(name, 'w') as f:
        # Store the waveforms, frequency, and applied field
        f.create_dataset('frequency', data=frequency)
        f.create_dataset('waveforms', data=waveforms)
        f.create_dataset('applied_field', data=applied_field)


    vxm.move_degree(step_size)
    
    
# Turn off all power
lna_psu.lna(state=False)
shno_bias.off()
magnet.off()
shno_bias.chirp(0)

## Field + Bias Sweep

In [ ]:
# initialize the parameteters
# Remember to create a folder with the name of the sample!

# Metadata
sample_name = "mm_L3R4"
date = time.localtime()[1:3]
date = f"{date[0]}-{date[1]}"


# Set bias current range
start_bias = 4e-3
stop_bias = 8e-3
bias_step = 2e-3

total_steps = int((stop_bias-start_bias)/bias_step)
bias_range = np.linspace(start_bias, stop_bias, total_steps)

#applied_field = np.zeros_like(bias_range)


# Set magnet range in output voltage
start_field = 2 #[A]
stop_field = 4.5  #[A]
step_size = 0.15



magnet_range = np.linspace(start_field, stop_field, int(abs((stop_field-start_field)/step_size)+1))
applied_field = np.zeros_like(magnet_range)



#applied_field = np.zeros(angle_steps)

# Turn on FSV
start_f = 3E9; stop_f = 9E9
bandwidth = 500E3
capture_points = int(2*(stop_f-start_f)/bandwidth)

waveforms = np.zeros((len(magnet_range), capture_points))
waveforms_bg = np.zeros_like(waveforms)
fsv.show_screen(start_f = start_f, stop_f = stop_f, bandwidth=bandwidth, points = capture_points)
frequency = np.linspace(start_f, stop_f, capture_points)


In [ ]:
# Start measurements

lna_psu.lna(state=True)
shno_bias.on()
time.sleep(1)




mag_R = magnet.set_calibrated_current(start_field)

for bias_current in tqdm(bias_range):
    # set the new bias current
    shno_bias.set_output(I=bias_current, V=2, turn_on=True) 
    

    
    
    progress_bar = tqdm(enumerate(magnet_range), total=len(magnet_range), leave=False)
    for i, field in progress_bar:
        
        mag_R = magnet.set_calibrated_current(field, mag_R)
        
        applied_field[i] = magnet.field_strength()
        progress_bar.set_postfix({'Applied Field': applied_field[i]})
        waveforms[i] = fsv.get_trace(avg_count=50)
    

    # Save the data
    #name = f'bias_sweep/field_sweep/{date}_{sample_name}_{str(np.round(bias_current*1000, 2)).replace(".", "_")}mA_82deg.h5'
    name = f"../Memristor_SHNO_matrix/bias_sweep/field_sweep_78_{sample_name.split('_')[-1]}/{date}_{sample_name}_" + "{:0.2f}".format((bias_current*1000)) + "_mA.h5"
    with h5py.File(name, 'w') as f:
        # Store the waveforms, frequency, and applied field
        f.create_dataset('frequency', data=frequency)
        f.create_dataset('waveforms', data=waveforms)
        f.create_dataset('applied_field', data=applied_field)
    
    
# Turn off all power
lna_psu.lna(state=False)
shno_bias.off()
magnet.off()
shno_bias.chirp(0)

## Saving data

In [ ]:
# Save the data
name = f'../Memristor_SHNO_matrix/field_sweep/{date}_{sample_name}_{str(bias_current*1000).replace(".", "_")}mA_77_deg.h5'
ans = input(f"Confirm saving? [y/n]:{name}")
if ans == 'y':
    with h5py.File(name, 'w') as f:
        # Store the waveforms, frequency, and applied field
        f.create_dataset('frequency', data=frequency)
        f.create_dataset('waveforms', data=waveforms)
        f.create_dataset('applied_field', data=applied_field)
        #f.create_dataset('angle', data=angle_range)
        #f.create_dataset('bias_range', data=bias_range)
    print(f"File saved as: '{name}'")
    print("Time:", time.strftime("%H:%M:%S", time.localtime()))
else: 
    print("Save cancelled!")

In [ ]:
# Load data
name = '../Memristor_SHNO_matrix/angle_sweep/field_sweep/X6_L1R2/3-13_2_0mA_80.0deg.h5'
ans = input("Confirm loading? [y/n]")
if ans == 'y':
    with h5py.File(name, 'r') as f:
        applied_field = f['applied_field'][:]
        waveforms = f['waveforms'][:]
        frequency = f['frequency'][:]
    print("Data loaded!")
    print("Time:", time.strftime("%H:%M:%S", time.localtime()))
else: 
    print("Loading cancelled!")
    

## Visualizations

In [ ]:
x_values = applied_field#bias_range#applied_field#applied_field#applied_field #bias_range#applied_field#bias_range*1000 # #applied_field #angle_range
y_values = frequency/1e9
X, Y = np.meshgrid(x_values, y_values)

plt.pcolormesh(X, Y, (waveforms.T), shading='nearest', cmap='viridis') #vmax = -74, vmin = -80
plt.colorbar(label='Amplitude [dBm]')
plt.xlabel('Applied field [mT]')
plt.ylabel('Frequency [GHz]')
plt.ylim(6.8, 7.2)
plt.tight_layout()
plt.show()

## Disconnect

In [ ]:
vxm.Close()
fsv.close()
lna_psu.close()
magnet.close()